In [213]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.mixture import GaussianMixture
import os
from feature_extraction import *
import warnings
warnings.filterwarnings('ignore')

In [214]:
TRAIN_PATH = "../data/processed/train2/"
# TEST_PATH = "../data/processed/test/"
REF_PATH = "../data/processed/reference/"

In [215]:
def setup_df(PATH):
    all_files = os.listdir(PATH)
    df = pd.DataFrame(all_files, columns=['file'])
    df['speaker'] = df['file'].apply(lambda x: int(x.split('.')[0].split('-')[-1]))
    # 0 for neutral, 1 for emotional
    df['speech_type'] = df['file'].apply(lambda x: 0 if int(x.split('.')[0].split('-')[2])==1 else 1)
    df['F0_contour'] = df['file'].apply(lambda x: get_F0_contour(PATH+x))
    return df

In [216]:
def get_audio_features(df, norm=False):
    if norm==False:
        df['SQ25'] = df['F0_contour'].apply(get_SQ25)
        df['SQ75'] = df['F0_contour'].apply(get_SQ75)
        df['IDR'] = df['F0_contour'].apply(get_IDR)
        df['voiced_segments'] = df['F0_contour'].apply(get_voiced_segments)
        df['F0_median'] = df['F0_contour'].apply(get_F0_median)
        df['sdmedian'] = df['F0_contour'].apply(get_sdmedian)
    else:
        df['SQ25'] = df['F0_contour_norm'].apply(get_SQ25)
        df['SQ75'] = df['F0_contour_norm'].apply(get_SQ75)
        df['IDR'] = df['F0_contour_norm'].apply(get_IDR)
        df['voiced_segments'] = df['F0_contour_norm'].apply(get_voiced_segments)
        df['F0_median'] = df['F0_contour_norm'].apply(get_F0_median)
        df['sdmedian'] = df['F0_contour_norm'].apply(get_sdmedian)
    
    df['SVMeanRange'] = df['voiced_segments'].apply(get_voiced_segment_range)
    df['SVMaxCurv'] = df['voiced_segments'].apply(get_max_voiced_curvature)
    
    return df

In [217]:
ref_df = setup_df(REF_PATH)
ref_df = get_audio_features(ref_df)
temp_df = setup_df(TRAIN_PATH)
# test_df = setup_df(TEST_PATH)

## Training the GMMs on the Reference Data

In [483]:
trained_GMMs = {}
features = [ 'SQ25', 'SQ75','F0_median', 'sdmedian', 'IDR', 'SVMeanRange','SVMaxCurv']
# features = [ 'SQ75','IDR', 'SVMeanRange','SVMaxCurv']

for feature in features:
    gmm = GaussianMixture(n_components=2)
    gmm.fit(ref_df[feature].values.reshape(-1,1))
    trained_GMMs[feature] = gmm

## IFN Algo

In [564]:
ref_df['F0_contour_sum'] = ref_df['F0_contour'].apply(sum)

In [565]:
ref_df['F0_contour_length'] = ref_df['F0_contour'].apply(len)

In [566]:
ref_df['F0_contour_mean'] = ref_df['F0_contour_sum']/ref_df['F0_contour_length']

In [567]:
avg_F0_ref = np.sum(ref_df['F0_contour_mean']) / 24

In [568]:
def stratified_sample_df(df):
    per_speaker_neutral_number = df[df['speech_type']==0]['speaker'].value_counts().values[0]
    sampled_df = []
    sampled_df.append(df[df['speech_type']==0])
    speakers = df['speaker'].unique()
    for speaker in speakers:
        speaker_df = df[df['speaker'] == speaker]
        speaker_emotional_df= speaker_df[speaker_df['speech_type']==1]
        sampled_df.append(speaker_emotional_df.sample(n=per_speaker_neutral_number))
    return pd.concat(sampled_df).reset_index(drop=True)

In [569]:
def infer_GMM(df):
    infered = np.zeros(shape=(1,len(features)))
    for i in range(len(features)):
        res = trained_GMMs[features[i]].score_samples([[df[features[i]]]])
        infered[:,i] = res
    return infered[0]

In [570]:
def get_changed_labels(neutral_list, emotional_list, row):
    if row['file'] in neutral_list:
        return 0
    else:
        return 1

In [571]:
def get_S_s_F0(F0_ref, df):
#     grouped_df_neu = df[df['speech_type']==0].groupby(['speaker']).sum().reset_index(level=0)
#     grouped_df_neu = grouped_df_neu[['speaker','F0_contour_sum','F0_contour_length']]
#     grouped_df_neu['F0_s_neu'] = grouped_df_neu['F0_contour_sum'] / grouped_df_neu['F0_contour_length']
#     grouped_df_neu['S_s_F0'] = F0_ref / grouped_df_neu['F0_s_neu']

    df_neu = df[df['changed_speech_type']==0]
    speakers = df['speaker'].unique()
    grouped_df_neu = {}
    UPPER_CAP = 100
    for speaker in speakers:
        speaker_df_neu = df_neu[df_neu['speaker']==speaker]
        speaker_mean = (speaker_df_neu['F0_contour_sum']/speaker_df_neu['F0_contour_length']).mean()
        if F0_ref/speaker_mean > UPPER_CAP:
            grouped_df_neu[speaker] = UPPER_CAP
        else:
            grouped_df_neu[speaker] = F0_ref/speaker_mean
    
    
        
        
    
#     return grouped_df_neu[['speaker','S_s_F0']].set_index('speaker').to_dict()['S_s_F0']
    return grouped_df_neu

def get_normalised_df(df, avg_F0_ref=avg_F0_ref, S0_func=get_S_s_F0):
    df_norm = S0_func(avg_F0_ref, df)
    df['F0_contour_norm'] = df.apply(lambda x: x['F0_contour']/df_norm[x['speaker']], axis=1)
#     df['F0_contour'] = df.apply(lambda x: x['F0_contour']/df_norm[x['speaker']], axis=1)

    return df_norm, df
        

def get_normalised_df_infer(df, df_norm):
    df['F0_contour_norm'] = df.apply(lambda x: x['F0_contour']/df_norm[x['speaker']], axis=1)
#     df['F0_contour'] = df.apply(lambda x: x['F0_contour']/df_norm[x['speaker']], axis=1)

    return df_norm, df

In [572]:
def get_pred_labels(df):
    
    grouped_sampled_df = df.groupby('speaker')
    neutral = []
    emotional = []

    for name, group in grouped_sampled_df:
        neu_result = group[group['predicted_likelihood'] >= 0.7]
        emo_result = group[group['predicted_likelihood'] < 0.7]

        total = group.shape[0]
        to_add = int(np.ceil(0.2 * total)) - neu_result.shape[0]
        converted_neu_add = neu_result
        if to_add > 0:
            emo_result_sort = emo_result.sort_values('predicted_likelihood', ascending=False)
            converted_neu_add = emo_result_sort.head(to_add)
            emo_result.drop(converted_neu_add.index, inplace=True)

        neutral.extend(converted_neu_add['file'].tolist())
        emotional.extend(emo_result['file'].tolist()) 

    df['changed_speech_type'] = df.apply(lambda x: \
                                         get_changed_labels(neutral,emotional,x), axis=1) 
    
    return df
    

In [573]:
def get_stopping_criteria(df, count):
    df = get_pred_labels(df)
    
    if count==0:
        return df, 1000000

    else:
        changed_dict = (df['prev_changed_speech_type'] != df['changed_speech_type']).\
                        value_counts().to_dict()

        if True not in changed_dict.keys():
            changed_dict[True] = 0
            epsilon = changed_dict[True]/changed_dict[False]
        elif False not in changed_dict.keys():
            epsilon = 1000000
        else:
            epsilon = changed_dict[True]/changed_dict[False]
    
    return df, epsilon

In [574]:
# for every epoch
epoch = 0


for iterations in range(400):

    sampled_df = stratified_sample_df(temp_df)
    
    train_df, test_df = train_test_split(sampled_df, test_size = 0.33, stratify = sampled_df[['speaker','speech_type']])
    
    train_df = get_audio_features(train_df)
    
    train_df['F0_contour_sum'] = train_df['F0_contour'].apply(sum)
    train_df['F0_contour_length'] = train_df['F0_contour'].apply(len)

    test_df['F0_contour_sum'] = test_df['F0_contour'].apply(sum)
    test_df['F0_contour_length'] = test_df['F0_contour'].apply(len)
    
    count = 0

    # for 400 iterations
    sampled_df_norm = None
    ldc_clf = None
    epsilon = 1000000
    max_iters = 1000
    
    ldc_clf = LinearDiscriminantAnalysis(solver='lsqr')
    
    print('*************************************************') 
    print('*************************************************') 

            
    for stage in ['train', 'test']:

        print('----------------------------------------')   
        print('Stage - ', stage)


        if stage == 'train':
            
            while epsilon > 0.05 and count < max_iters:
        
                print('=========================================')   
                
                print(sampled_df_norm)

                if count ==0 :
                    train_df['F0_contour_sum'] = train_df['F0_contour'].apply(sum)
                    train_df['F0_contour_length'] = train_df['F0_contour'].apply(len)
                    train_df = get_audio_features(train_df, norm=False)
                else:
                    train_df['F0_contour_sum'] = train_df['F0_contour_norm'].apply(sum)
                    train_df['F0_contour_length'] = train_df['F0_contour_norm'].apply(len)
                    # Change above to F0_contour_norm when norm=True
                    train_df = get_audio_features(train_df, norm=True)
                train_df['inferred'] = train_df.apply(infer_GMM,axis=1)
                
                ldc_clf.fit(np.array(train_df['inferred'].tolist()),train_df['speech_type'].values)

                train_df['predicted_likelihood'] = ldc_clf.predict_proba(np.array(train_df['inferred'].tolist()))[:,0]

                train_df, epsilon = get_stopping_criteria(train_df, count)
                
                print(count)
                print(epsilon)
                
                train_df['prev_changed_speech_type'] = train_df['changed_speech_type']
                                
                sampled_df_norm, train_df = get_normalised_df(train_df, avg_F0_ref=avg_F0_ref, S0_func=get_S_s_F0)                
                count+=1

        else:
                sampled_df_test = test_df
                if count!=0:
                    _, sampled_df_test = get_normalised_df_infer(test_df, sampled_df_norm)
                if count == 0 :
                    sampled_df_test = get_audio_features(sampled_df_test, norm=False)
                else:
                    sampled_df_test = get_audio_features(sampled_df_test, norm=True)
                sampled_df_test['inferred'] = sampled_df_test.apply(infer_GMM,axis=1)

                sampled_df_test['predicted_likelihood'] = ldc_clf.predict_proba(np.array(sampled_df_test['inferred'].tolist()))[:,0]
#                 sampled_df_test['changed_speech_type'] = sampled_df_test['predicted_likelihood'].\
#                                                             apply(lambda x: 0 if x >=0.7 else 1)

                sampled_df_test = get_pred_labels(sampled_df_test)

                print(classification_report(sampled_df_test['speech_type'],sampled_df_test['changed_speech_type']))
                
#         count+=1



*************************************************
*************************************************
----------------------------------------
Stage -  train
None
0
1000000
{3: 0.9450852856071655, 1: 1.5484529257396515, 10: 0.6522133840547156, 6: 0.5876222003377473, 7: 1.1982127829733265, 5: 2.066578836469, 22: 0.819849467761128, 9: 1.322744057077813, 14: 0.8022194874556691, 24: 0.6580974231835958, 12: 0.7157602667654495, 18: 0.6648433262961593, 20: 0.6634112456833592, 23: 0.8139034595791711, 21: 1.8516326959757745, 11: 1.3835581566432795, 4: 0.5905564130512672, 19: 1.1169408619433092, 17: 1.246778584044872, 16: 0.8011136270991973, 2: 0.8230426735719764, 13: 1.5174202995180461, 15: 1.381951366066339, 8: 0.6641805051926852}
1
0.37142857142857144
{3: 0.8931861970711779, 1: 2.4728743123203816, 10: 0.42538229834010444, 6: 0.34529985032977567, 7: 1.1953681237548925, 5: 4.052257525838966, 22: 0.6721531497882044, 9: 1.749651840534674, 14: 0.6435561060536367, 24: 0.43309221840088896, 12: 0.51231

13
0.06666666666666667
{3: 0.9660343024642741, 1: 67.03912419240525, 10: 0.0031755007617560596, 6: 0.0009030642002500425, 7: 2.0443392892089394, 5: 78.63028609320885, 22: 0.02872715634916062, 9: 100, 14: 0.036473139384154446, 24: 0.007829435105146351, 12: 0.012093689001378357, 18: 0.02795113802795455, 20: 0.001319904088993354, 23: 1.8809393393691112, 21: 2.7251979008573, 11: 5.442312488980862, 4: 0.002421544149770258, 19: 1.745567893472984, 17: 1.980425615524737, 16: 0.21086648495078883, 2: 0.026247747048803827, 13: 2.7789879343209445, 15: 2.1210949257762683, 8: 0.012617622273391301}
14
0.06666666666666667
{3: 0.9129848046507674, 1: 78.38832540570829, 10: 0.002071104097893248, 6: 0.0005787427988054315, 7: 2.462258113928388, 5: 94.10750997322324, 22: 0.023551943843150045, 9: 100, 14: 0.029259463182655595, 24: 0.005883981136235128, 12: 0.008656182065804955, 18: 0.018583127580268373, 20: 0.0008756392158616401, 23: 2.394610340216946, 21: 2.5639395902437063, 11: 4.502597034227092, 4: 0.0016

1
0.3333333333333333
{6: 0.4107087431994231, 5: 3.3260499821084824, 17: 2.3213806235413905, 19: 0.7348595784632646, 3: 1.463077829839845, 23: 1.3626620665958995, 20: 0.6672701746457992, 2: 0.6081066736821077, 8: 0.6134742239175939, 10: 0.5359641494286186, 14: 0.6435561060536367, 7: 1.5948781698192527, 15: 1.8878335658961432, 16: 0.6417830435240311, 18: 0.7065432468938843, 4: 0.7680258697944902, 12: 0.5123127594801474, 9: 3.4491408964633132, 1: 2.245443127935743, 22: 0.7404238099102959, 11: 1.6810697682330547, 21: 1.5759814822555516, 24: 0.5761813637197617, 13: 1.5468609628903254}
2
0.45454545454545453
{6: 0.26320911344650993, 5: 2.3628136098831165, 17: 2.487881221673735, 19: 1.0411075631667823, 3: 1.695074022797912, 23: 1.5043688335902852, 20: 0.545070364508454, 2: 0.4091788039610876, 8: 0.5026675191423582, 10: 0.3923770621346138, 14: 0.516273249547315, 7: 1.2259248184669032, 15: 2.543855471185225, 16: 0.5918336037689328, 18: 0.6109965622970099, 4: 0.6730754856767228, 12: 0.36669311739

6
0.14285714285714285
{3: 1.4685414834436008, 8: 0.162505737227287, 7: 1.3270092149132233, 19: 1.3710300291005728, 9: 100, 14: 0.2662534079068233, 12: 0.13643326131301492, 1: 1.162008312922273, 2: 0.1967662366712624, 13: 4.232276032000331, 22: 0.26130330475499197, 11: 1.776564070935274, 5: 1.3427602995419459, 6: 0.06732880580894092, 21: 1.180802485727177, 15: 2.2579101188704858, 16: 0.22462958528271737, 17: 1.490476230390552, 4: 0.3970228210446584, 18: 0.2544204854497533, 10: 0.08076392293915181, 24: 0.08389384568087829, 20: 1.0071700013116056, 23: 0.9535233739604935}
7
0.09090909090909091
{3: 1.357777490069339, 8: 0.12535326641416042, 7: 1.2269656654683192, 19: 1.3492835528358358, 9: 100, 14: 0.22039131996253644, 12: 0.1030637503215559, 1: 1.4327921140665234, 2: 0.15598627118746217, 13: 5.061732118789464, 22: 0.2142293753276039, 11: 1.1997122724268403, 5: 1.2942977241613762, 6: 0.045793205166366806, 21: 1.095504909916742, 15: 2.3842694750768296, 16: 0.179953821819626, 17: 1.2446718860

20
1.0
{3: 1.8852334787057425, 8: 0.0024942649100577995, 7: 1.2991518122171757, 19: 1.6806905200482989, 9: 100, 14: 0.008359890936289065, 12: 0.0013957895522597043, 1: 2.0903281282747743, 2: 0.003964693368402675, 13: 65.78200640450638, 22: 0.010595640552641407, 11: 1.9073831489976036, 5: 3.3904544117973647, 6: 0.00025589884554296606, 21: 1.8031605462263618, 15: 7.463699084502552, 16: 0.008501009679471103, 17: 1.1919858897672235, 4: 0.04637693593215726, 18: 0.008151295188096649, 10: 0.0003793113332588396, 24: 0.00048612533703837113, 20: 0.6499092355076213, 23: 0.6863490671732309}
21
0.06666666666666667
{3: 1.7430407038345521, 8: 0.0019240197860870753, 7: 1.2012084391785152, 19: 1.654032390228504, 9: 100, 14: 0.006919901655630133, 12: 0.0010544005510905975, 1: 1.5405563664513429, 2: 0.0031430073848086925, 13: 78.67419141343905, 22: 0.00868683026767128, 11: 1.2880542894621831, 5: 3.2680869627730185, 6: 0.00017404776744501987, 21: 1.6729057193190393, 15: 7.881389852329666, 16: 0.0068102746

...

9
0.2972972972972973
{4: 0.023131619830663152, 3: 2.1385405013926486, 7: 3.0116601101806397, 2: 0.06411798823107186, 24: 0.05746566403736735, 5: 4.059727415815338, 17: 6.317528518920891, 14: 0.03639785820865006, 21: 1.7556661674525065, 18: 0.14089489502212607, 22: 0.20201784872355755, 13: 3.902483852154157, 15: 3.2511946199443034, 16: 0.04125289612668817, 1: 21.565526163349993, 12: 0.0573374950013847, 19: 0.055650447127138646, 11: 3.5618178619126666, 9: 4.411551110580878, 6: 0.011686103845420062, 23: 5.174101327619601, 20: 0.04837399314393619, 10: 0.04422610613517576, 8: 0.07458810421151645}
10
0.11627906976744186
{4: 0.015871817115584472, 3: 2.6293598503806725, 7: 3.3303060871401344, 2: 0.04840106325803034, 24: 0.04318662568067539, 5: 3.9766505916824673, 17: 9.625435511127469, 14: 0.026132647581511983, 21: 1.4795014771417185, 18: 0.11599719350908125, 22: 0.17383208129082686, 13: 3.290936668997618, 15: 2.1417395849315377, 16: 0.02431965459791767, 1: 29.634150307700803, 12: 0.0433136114

22
0.23076923076923078
{4: 0.0001728517997777162, 3: 4.952500226987828, 7: 7.149154389341136, 2: 0.0016571242736821048, 24: 0.001300507470762503, 5: 45.24628737388512, 17: 74.50437219238907, 14: 0.0004903141269108655, 21: 3.1187113253046967, 18: 0.010909885817202059, 22: 0.01945197194220452, 13: 3.636332737425288, 15: 2.204266223009004, 16: 0.00016402807260575467, 1: 75.8970028746756, 12: 0.001495727809474375, 19: 6.750528825680982e-05, 11: 32.65566134082879, 9: 62.247334956212576, 6: 3.594515865425277e-05, 23: 11.84029692297615, 20: 0.001022257791350799, 10: 0.0007674693119438782, 8: 0.002553524379198452}
23
0.23076923076923078
{4: 0.00011860268214052199, 3: 2.4920379970922606, 7: 6.070126785636142, 2: 0.001292245307341582, 24: 0.0009773580498125083, 5: 44.32038337250265, 17: 62.21736084331281, 14: 0.00035203187531933675, 21: 2.628140872169381, 18: 0.008981987147946218, 22: 0.016738009979263162, 13: 3.953148611819884, 15: 1.452070631695001, 16: 0.000151261904329046, 1: 66.120623366445

35
0.17073170731707318
{4: 1.3985135154648814e-06, 3: 2.03862832396065, 7: 9.186091588742864, 2: 4.721444409591345e-05, 24: 2.0238792165417564e-05, 5: 100, 17: 83.50833516542075, 14: 7.380038178334937e-06, 21: 2.7514641582984707, 18: 0.0008256665284941848, 22: 0.0017845612480015497, 13: 3.2627818243105216, 15: 4.497196796682994, 16: 1.0202118013589663e-06, 1: 100, 12: 3.3190373302288386e-05, 19: 2.520569257968951e-07, 11: 94.07537304158006, 9: 62.247334956212576, 6: 1.1056331928675933e-07, 23: 92.5778910614944, 20: 2.2671523335741073e-05, 10: 1.1120402860601188e-05, 8: 8.741992874185534e-05}
36
0.14285714285714285
{4: 9.59593444541552e-07, 3: 2.5065166927539035, 7: 10.15801770976812, 2: 3.56410011983441e-05, 24: 1.5209867598650245e-05, 5: 97.95363541381545, 17: 69.73642042100249, 14: 5.298661689019298e-06, 21: 2.3186613503020665, 18: 0.0006797620315815622, 22: 0.0014630715893612038, 13: 2.751480532747194, 15: 2.962549316979455, 16: 6.014413763720723e-07, 1: 100, 12: 2.5072510282212037e

3
0.09090909090909091
{14: 0.4694585391304925, 9: 1.3558162515274417, 23: 1.90707562638316, 21: 2.2461162150672025, 3: 1.334189251255528, 6: 0.18557962424207475, 24: 0.3286848250022348, 12: 0.30854934529117833, 7: 1.6565234647356832, 10: 0.2872575694727423, 2: 0.3916854467313041, 8: 0.4552430465032464, 1: 1.8197873433252376, 22: 0.4153932547124793, 16: 0.4888880168183056, 18: 0.7274399282488836, 19: 1.0034101915013525, 5: 7.854838918143955, 17: 1.3678699916088375, 13: 4.118629723571064, 15: 2.7116739685047553, 11: 1.8558627424990926, 20: 0.33163863657942777, 4: 0.5898637366735826}
4
0.09090909090909091
{14: 0.388594414321492, 9: 1.4630226309246257, 23: 2.110303581019752, 21: 2.7497314369895682, 3: 1.3301631656947097, 6: 0.12180436830286574, 24: 0.24949351247644166, 12: 0.2330828448937473, 7: 1.4816162311128618, 10: 0.2103000383622785, 2: 0.309864479062889, 8: 0.38185818942777905, 1: 1.5853784599863354, 22: 0.34055993878758867, 16: 0.4066103631584687, 18: 0.7639963595005934, 19: 1.52693

KeyboardInterrupt: 

In [ ]:
ref_df['F0_contour_length'].sum()/24